# Music Genre Prediction

In [1]:
class Config:
    NB = '103'

    raw_data_dir = '../data/raw/'
    processed_data_dir = '../data/processed/'
    interim_dir = '../data/interim/'
    submission_dir = '../data/submission/'

    random_seed = 42
    n_folds = 5

    row_id = 'index'
    target = 'genre'

## Import libralies

In [2]:
import gc
import warnings
warnings.filterwarnings('ignore')

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plotly_template = dict(
    layout=go.Layout(
        template='plotly_dark',
        font=dict(
            family="Franklin Gothic",
            size=12
        ),
        height=500,
        width=1000,
    )
)

color_palette = {
    'Bin': ['#016CC9','#E876A3'],
    'Cat5': ['#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E'],
    'Cat10': ['#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E', '#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E'],
}

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import make_pipeline

## Load data

In [5]:
df_train = pd.read_csv(Config.raw_data_dir + 'train.csv')
df_test = pd.read_csv(Config.raw_data_dir + 'test.csv')

## 欠損値補完
- objectで同じ条件になっているものの中央値で補完する

In [6]:
display(df_train.info())
display(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             4046 non-null   int64  
 1   genre             4046 non-null   int64  
 2   popularity        4046 non-null   int64  
 3   duration_ms       4046 non-null   int64  
 4   acousticness      4046 non-null   float64
 5   positiveness      4036 non-null   float64
 6   danceability      4038 non-null   float64
 7   loudness          4046 non-null   float64
 8   energy            4046 non-null   float64
 9   liveness          4043 non-null   float64
 10  speechiness       4038 non-null   float64
 11  instrumentalness  4045 non-null   float64
 12  tempo             4046 non-null   object 
 13  region            4046 non-null   object 
dtypes: float64(8), int64(4), object(2)
memory usage: 442.7+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             4046 non-null   int64  
 1   popularity        4046 non-null   int64  
 2   duration_ms       4046 non-null   int64  
 3   acousticness      4045 non-null   float64
 4   positiveness      4032 non-null   float64
 5   danceability      4035 non-null   float64
 6   loudness          4046 non-null   float64
 7   energy            4045 non-null   float64
 8   liveness          4040 non-null   float64
 9   speechiness       4035 non-null   float64
 10  instrumentalness  4044 non-null   float64
 11  tempo             4046 non-null   object 
 12  region            4046 non-null   object 
dtypes: float64(8), int64(3), object(2)
memory usage: 411.0+ KB


None

In [7]:
col_list = ['acousticness', 'positiveness', 'danceability', 'energy', 'liveness', 'speechiness', 'instrumentalness']

imputer = KNNImputer(n_neighbors=3)
imputer.fit(df_train[col_list])
for df in [df_train, df_test]:
    df[col_list] = imputer.transform(df[col_list])

In [8]:
display(df_train.info())
display(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             4046 non-null   int64  
 1   genre             4046 non-null   int64  
 2   popularity        4046 non-null   int64  
 3   duration_ms       4046 non-null   int64  
 4   acousticness      4046 non-null   float64
 5   positiveness      4046 non-null   float64
 6   danceability      4046 non-null   float64
 7   loudness          4046 non-null   float64
 8   energy            4046 non-null   float64
 9   liveness          4046 non-null   float64
 10  speechiness       4046 non-null   float64
 11  instrumentalness  4046 non-null   float64
 12  tempo             4046 non-null   object 
 13  region            4046 non-null   object 
dtypes: float64(8), int64(4), object(2)
memory usage: 442.7+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             4046 non-null   int64  
 1   popularity        4046 non-null   int64  
 2   duration_ms       4046 non-null   int64  
 3   acousticness      4046 non-null   float64
 4   positiveness      4046 non-null   float64
 5   danceability      4046 non-null   float64
 6   loudness          4046 non-null   float64
 7   energy            4046 non-null   float64
 8   liveness          4046 non-null   float64
 9   speechiness       4046 non-null   float64
 10  instrumentalness  4046 non-null   float64
 11  tempo             4046 non-null   object 
 12  region            4046 non-null   object 
dtypes: float64(8), int64(3), object(2)
memory usage: 411.0+ KB


None

## TrainとTestの統合

In [9]:
df_dataset = pd.concat(objs=[df_train, df_test], axis=0).reset_index(drop=True)
df_dataset

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region
0,0,10.0,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,121-152,region_H
1,1,8.0,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,153-176,region_I
2,2,3.0,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,64-76,region_E
3,3,10.0,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,177-192,region_C
4,4,3.0,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,97-120,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087,8087,NaN,34,366668,0.117208,0.316026,0.502289,-2.909329,0.824708,0.320241,0.206592,0.198859,121-152,region_B
8088,8088,NaN,53,240081,0.225540,0.391226,0.483655,-5.013634,0.630808,0.163884,0.205218,0.179438,121-152,region_I
8089,8089,NaN,48,217839,0.215571,0.828567,0.689747,-8.974036,0.697981,0.318250,0.123748,0.135281,121-152,region_H
8090,8090,NaN,45,249612,0.205074,0.730554,0.580581,-1.915320,0.828423,0.360663,0.165837,0.162821,121-152,region_B


## 正規化＆標準化

In [10]:
def clip_outlier(df, col):
    '''clip outlier
    '''

    Q1 = np.percentile(df[col], 25)
    Q3 = np.percentile(df[col], 75)

    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    df.loc[:, col] = np.maximum(df[col], Q1 - outlier_step)
    df.loc[:, col] = np.minimum(df[col], Q3 + outlier_step)

    return df


def clip_manual_outlier(df, col, quantile, upper=True):
    '''clip manual outlier

    quantile: 0 ~ 1 e.g.)0.01
    '''

    Q = df[col].quantile(quantile)

    if upper:
        df.loc[:, col] = np.minimum(df[col], Q)
    else:
        df.loc[:, col] = np.maximum(df[col], Q)

    return df


def preprocess_dataset(df):
    """Preprocess the data (select columns and scale)
    ### MinMaxScaler: 正規化（値を0~1の範囲に）
    ### RobustScalar: 正規化（ただし、四分位範囲を分母とする）
    ### StandardScaler: 標準化（平均との差をとり標準偏差で割る）
    """
    # preproc = make_pipeline(MinMaxScaler(), StandardScaler(with_std=False))
    preproc = make_pipeline(RobustScaler(), StandardScaler(with_std=False))
    df_f = pd.DataFrame(preproc.fit_transform(df), columns=df.columns, index=df.index)

    return df_f

def normalize_dataset(df):
    """Preprocess the data (select columns and scale)
    ### MinMaxScaler: 正規化（値を0~1の範囲に）
    """
    preproc = make_pipeline(MinMaxScaler())
    df_f = pd.DataFrame(preproc.fit_transform(df), columns=df.columns, index=df.index)

    return df_f

def standardize_dataset(df):
    """Preprocess the data (select columns and scale)
    ### StandardScaler: 標準化（平均との差をとり標準偏差で割る）
    """
    preproc = make_pipeline(StandardScaler(with_std=False))
    df_f = pd.DataFrame(preproc.fit_transform(df), columns=df.columns, index=df.index)

    return df_f

In [11]:
col_list = ['popularity', 'duration_ms', 'acousticness', 'positiveness', 'danceability', 'loudness', 'energy', 'liveness', 'speechiness', 'instrumentalness']

for col in col_list:
    df_dataset = clip_manual_outlier(df_dataset, col, 0.99, True)
    df_dataset = clip_manual_outlier(df_dataset, col, 0.01, False)

df_dataset.describe()

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness
count,8092.000000,4046.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000
mean,4045.500000,7.281760,41.103188,240561.944827,0.340755,0.465519,0.501603,-7.646038,0.606332,0.267230,0.199378,0.215153
std,2336.103522,2.887542,16.068290,69374.790084,0.237945,0.221872,0.158076,3.896445,0.199625,0.154353,0.079634,0.153964
min,0.000000,0.000000,2.000000,55300.570000,0.054800,0.065861,0.140022,-20.426182,0.109110,0.075882,0.067761,0.080785
25%,2022.750000,7.000000,31.000000,203105.750000,0.147911,0.281609,0.390672,-9.769150,0.461977,0.168740,0.149444,0.143730
50%,4045.500000,8.000000,42.000000,235738.500000,0.249046,0.447637,0.509021,-7.145340,0.635327,0.219955,0.184712,0.172111
75%,6068.250000,10.000000,52.000000,272485.500000,0.508564,0.642323,0.616651,-4.906935,0.770725,0.321223,0.227209,0.205558
max,8091.000000,10.000000,78.000000,488723.770000,0.901917,0.915698,0.840459,-0.656288,0.929654,0.885348,0.497604,0.850310


In [12]:
df_dataset[col_list] = normalize_dataset(df_dataset[col_list])
df_dataset.describe()

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness
count,8092.000000,4046.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000
mean,4045.500000,7.281760,0.514516,0.427438,0.337562,0.470275,0.516222,0.646445,0.605966,0.236388,0.306199,0.174611
std,2336.103522,2.887542,0.211425,0.160062,0.280888,0.261076,0.225682,0.197090,0.243284,0.190685,0.185262,0.200077
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2022.750000,7.000000,0.381579,0.341018,0.109915,0.253870,0.357848,0.539054,0.430039,0.114715,0.190030,0.081796
50%,4045.500000,8.000000,0.526316,0.416309,0.229302,0.449234,0.526813,0.671771,0.641302,0.177985,0.272079,0.118678
75%,6068.250000,10.000000,0.657895,0.501092,0.535657,0.678321,0.680474,0.784994,0.806312,0.303090,0.370945,0.162142
max,8091.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## tempoをEncodingする

In [13]:
df_dataset['tempo_int'] = df_dataset['tempo'].str.split('-', expand=True)[1]
df_dataset = df_dataset.astype({'tempo_int': int})
df_dataset

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region,tempo_int
0,0,10.0,0.118421,0.336377,0.068480,0.107533,0.068271,0.937857,0.956448,0.355402,0.749919,1.000000,121-152,region_H,152
1,1,8.0,0.881579,0.584169,0.054931,0.330301,0.591662,0.752644,0.932672,0.145787,0.218070,0.056041,153-176,region_I,176
2,2,3.0,0.539474,0.327450,0.521320,0.234785,0.453460,0.565026,0.403174,0.174515,0.700945,0.111347,64-76,region_E,76
3,3,10.0,0.565789,0.567093,0.130875,0.211419,0.309173,0.775795,0.925719,0.372026,0.369707,0.122951,177-192,region_C,192
4,4,3.0,0.723684,0.512311,0.160450,0.837474,0.985751,0.834212,0.659366,0.115435,0.359962,0.188746,97-120,unknown,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087,8087,NaN,0.421053,0.718391,0.073671,0.294368,0.517201,0.886037,0.872101,0.301877,0.322981,0.153437,121-152,region_B,152
8088,8088,NaN,0.671053,0.426328,0.201554,0.382856,0.490598,0.779597,0.635794,0.108716,0.319784,0.128200,121-152,region_I,152
8089,8089,NaN,0.605263,0.375011,0.189786,0.897473,0.784831,0.579272,0.717659,0.299417,0.130252,0.070818,121-152,region_H,152
8090,8090,NaN,0.565789,0.448318,0.177394,0.782141,0.628978,0.936316,0.876629,0.351814,0.228167,0.106605,121-152,region_B,152


In [14]:
df_dataset.drop(columns=['tempo'], inplace=True)
df_dataset

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,region,tempo_int
0,0,10.0,0.118421,0.336377,0.068480,0.107533,0.068271,0.937857,0.956448,0.355402,0.749919,1.000000,region_H,152
1,1,8.0,0.881579,0.584169,0.054931,0.330301,0.591662,0.752644,0.932672,0.145787,0.218070,0.056041,region_I,176
2,2,3.0,0.539474,0.327450,0.521320,0.234785,0.453460,0.565026,0.403174,0.174515,0.700945,0.111347,region_E,76
3,3,10.0,0.565789,0.567093,0.130875,0.211419,0.309173,0.775795,0.925719,0.372026,0.369707,0.122951,region_C,192
4,4,3.0,0.723684,0.512311,0.160450,0.837474,0.985751,0.834212,0.659366,0.115435,0.359962,0.188746,unknown,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087,8087,NaN,0.421053,0.718391,0.073671,0.294368,0.517201,0.886037,0.872101,0.301877,0.322981,0.153437,region_B,152
8088,8088,NaN,0.671053,0.426328,0.201554,0.382856,0.490598,0.779597,0.635794,0.108716,0.319784,0.128200,region_I,152
8089,8089,NaN,0.605263,0.375011,0.189786,0.897473,0.784831,0.579272,0.717659,0.299417,0.130252,0.070818,region_H,152
8090,8090,NaN,0.565789,0.448318,0.177394,0.782141,0.628978,0.936316,0.876629,0.351814,0.228167,0.106605,region_B,152


## regionをone-hotEncodingする

In [15]:
col_list = ['region']
output_col_list = [i for i in sorted(df_dataset['region'].unique())]

ohe = OneHotEncoder(sparse=False)
ohe.fit(df_dataset[col_list])

df_dataset[output_col_list] = ohe.transform(df_dataset[col_list])
df_dataset.drop(columns=col_list, inplace=True)
df_dataset

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo_int,region_A,region_B,region_C,region_D,region_E,region_F,region_G,region_H,region_I,region_J,region_K,region_L,region_M,region_N,region_O,region_P,region_Q,region_R,region_S,region_T,unknown
0,0,10.0,0.118421,0.336377,0.068480,0.107533,0.068271,0.937857,0.956448,0.355402,0.749919,1.000000,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,8.0,0.881579,0.584169,0.054931,0.330301,0.591662,0.752644,0.932672,0.145787,0.218070,0.056041,176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3.0,0.539474,0.327450,0.521320,0.234785,0.453460,0.565026,0.403174,0.174515,0.700945,0.111347,76,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,10.0,0.565789,0.567093,0.130875,0.211419,0.309173,0.775795,0.925719,0.372026,0.369707,0.122951,192,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,3.0,0.723684,0.512311,0.160450,0.837474,0.985751,0.834212,0.659366,0.115435,0.359962,0.188746,120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087,8087,NaN,0.421053,0.718391,0.073671,0.294368,0.517201,0.886037,0.872101,0.301877,0.322981,0.153437,152,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8088,8088,NaN,0.671053,0.426328,0.201554,0.382856,0.490598,0.779597,0.635794,0.108716,0.319784,0.128200,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8089,8089,NaN,0.605263,0.375011,0.189786,0.897473,0.784831,0.579272,0.717659,0.299417,0.130252,0.070818,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8090,8090,NaN,0.565789,0.448318,0.177394,0.782141,0.628978,0.936316,0.876629,0.351814,0.228167,0.106605,152,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## duration_msでflagを作成する
- 600000以上であれば全てTarget_10になる

In [16]:
df_dataset['duration_long'] = np.where(df_dataset['duration_ms'] > 600000, 1, 0)
df_dataset

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo_int,region_A,region_B,region_C,region_D,region_E,region_F,region_G,region_H,region_I,region_J,region_K,region_L,region_M,region_N,region_O,region_P,region_Q,region_R,region_S,region_T,unknown,duration_long
0,0,10.0,0.118421,0.336377,0.068480,0.107533,0.068271,0.937857,0.956448,0.355402,0.749919,1.000000,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,8.0,0.881579,0.584169,0.054931,0.330301,0.591662,0.752644,0.932672,0.145787,0.218070,0.056041,176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,3.0,0.539474,0.327450,0.521320,0.234785,0.453460,0.565026,0.403174,0.174515,0.700945,0.111347,76,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,10.0,0.565789,0.567093,0.130875,0.211419,0.309173,0.775795,0.925719,0.372026,0.369707,0.122951,192,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,3.0,0.723684,0.512311,0.160450,0.837474,0.985751,0.834212,0.659366,0.115435,0.359962,0.188746,120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087,8087,NaN,0.421053,0.718391,0.073671,0.294368,0.517201,0.886037,0.872101,0.301877,0.322981,0.153437,152,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8088,8088,NaN,0.671053,0.426328,0.201554,0.382856,0.490598,0.779597,0.635794,0.108716,0.319784,0.128200,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8089,8089,NaN,0.605263,0.375011,0.189786,0.897473,0.784831,0.579272,0.717659,0.299417,0.130252,0.070818,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8090,8090,NaN,0.565789,0.448318,0.177394,0.782141,0.628978,0.936316,0.876629,0.351814,0.228167,0.106605,152,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## 各列の値を加減乗除

In [17]:
col_list = ['popularity', 'duration_ms', 'acousticness', 'positiveness', 'danceability', 'loudness', 'energy', 'liveness', 'speechiness', 'instrumentalness', 'tempo_int']

for i, col_i in enumerate(col_list):
    for j, col_j in enumerate(col_list):
        if i >= j:
            continue

        df_dataset[f'{col_i}_add_{col_j}'] = df_dataset[col_i] * df_dataset[col_j]
        df_dataset[f'{col_i}_sub_{col_j}'] = (df_dataset[col_i] - df_dataset[col_j]).abs()
        df_dataset[f'{col_i}_mul_{col_j}'] = df_dataset[col_i] * df_dataset[col_j]
        df_dataset[f'{col_i}_bigger_{col_j}'] = np.where(df_dataset[col_i] > df_dataset[col_j], 1, 0)

df_dataset

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo_int,region_A,region_B,region_C,region_D,region_E,region_F,region_G,region_H,region_I,region_J,region_K,region_L,region_M,region_N,region_O,region_P,region_Q,region_R,region_S,region_T,unknown,duration_long,popularity_add_duration_ms,popularity_sub_duration_ms,popularity_mul_duration_ms,popularity_bigger_duration_ms,popularity_add_acousticness,popularity_sub_acousticness,popularity_mul_acousticness,popularity_bigger_acousticness,popularity_add_positiveness,popularity_sub_positiveness,popularity_mul_positiveness,popularity_bigger_positiveness,popularity_add_danceability,popularity_sub_danceability,popularity_mul_danceability,popularity_bigger_danceability,popularity_add_loudness,popularity_sub_loudness,popularity_mul_loudness,popularity_bigger_loudness,popularity_add_energy,popularity_sub_energy,popularity_mul_energy,popularity_bigger_energy,popularity_add_liveness,popularity_sub_liveness,popularity_mul_liveness,popularity_bigger_liveness,popularity_add_speechiness,popularity_sub_speechiness,popularity_mul_speechiness,popularity_bigger_speechiness,popularity_add_instrumentalness,popularity_sub_instrumentalness,popularity_mul_instrumentalness,popularity_bigger_instrumentalness,popularity_add_tempo_int,popularity_sub_tempo_int,popularity_mul_tempo_int,popularity_bigger_tempo_int,duration_ms_add_acousticness,duration_ms_sub_acousticness,duration_ms_mul_acousticness,duration_ms_bigger_acousticness,duration_ms_add_positiveness,duration_ms_sub_positiveness,duration_ms_mul_positiveness,duration_ms_bigger_positiveness,duration_ms_add_danceability,duration_ms_sub_danceability,duration_ms_mul_danceability,duration_ms_bigger_danceability,duration_ms_add_loudness,duration_ms_sub_loudness,duration_ms_mul_loudness,duration_ms_bigger_loudness,duration_ms_add_energy,duration_ms_sub_energy,duration_ms_mul_energy,duration_ms_bigger_energy,duration_ms_add_liveness,duration_ms_sub_liveness,duration_ms_mul_liveness,duration_ms_bigger_liveness,duration_ms_add_speechiness,duration_ms_sub_speechiness,duration_ms_mul_speechiness,duration_ms_bigger_speechiness,duration_ms_add_instrumentalness,duration_ms_sub_instrumentalness,duration_ms_mul_instrumentalness,duration_ms_bigger_instrumentalness,duration_ms_add_tempo_int,duration_ms_sub_tempo_int,duration_ms_mul_tempo_int,duration_ms_bigger_tempo_int,acousticness_add_positiveness,acousticness_sub_positiveness,acousticness_mul_positiveness,acousticness_bigger_positiveness,acousticness_add_danceability,acousticness_sub_danceability,acousticness_mul_danceability,acousticness_bigger_danceability,acousticness_add_loudness,acousticness_sub_loudness,acousticness_mul_loudness,acousticness_bigger_loudness,acousticness_add_energy,acousticness_sub_energy,acousticness_mul_energy,acousticness_bigger_energy,acousticness_add_liveness,acousticness_sub_liveness,acousticness_mul_liveness,acousticness_bigger_liveness,acousticness_add_speechiness,acousticness_sub_speechiness,acousticness_mul_speechiness,acousticness_bigger_speechiness,acousticness_add_instrumentalness,acousticness_sub_instrumentalness,acousticness_mul_instrumentalness,acousticness_bigger_instrumentalness,acousticness_add_tempo_int,acousticness_sub_tempo_int,acousticness_mul_tempo_int,acousticness_bigger_tempo_int,positiveness_add_danceability,positiveness_sub_danceability,positiveness_mul_danceability,positiveness_bigger_danceability,positiveness_add_loudness,positiveness_sub_loudness,positiveness_mul_loudness,positiveness_bigger_loudness,positiveness_add_energy,positiveness_sub_energy,positiveness_mul_energy,positiveness_bigger_energy,positiveness_add_liveness,positiveness_sub_liveness,positiveness_mul_liveness,positiveness_bigger_liveness,positiveness_add_speechiness,positiveness_sub_speechiness,positiveness_mul_speechiness,positiveness_bigger_speechiness,positiveness_add_instrumentalness,positiveness_sub_instrumentalness,pos

## PCAで特徴量生成

In [18]:
train_len = len(df_train)
df_train = df_dataset[:train_len]
df_test = df_dataset[train_len:]
df_test.drop(columns=[Config.target], inplace=True)

df_test

,index,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo_int,region_A,region_B,region_C,region_D,region_E,region_F,region_G,region_H,region_I,region_J,region_K,region_L,region_M,region_N,region_O,region_P,region_Q,region_R,region_S,region_T,unknown,duration_long,popularity_add_duration_ms,popularity_sub_duration_ms,popularity_mul_duration_ms,popularity_bigger_duration_ms,popularity_add_acousticness,popularity_sub_acousticness,popularity_mul_acousticness,popularity_bigger_acousticness,popularity_add_positiveness,popularity_sub_positiveness,popularity_mul_positiveness,popularity_bigger_positiveness,popularity_add_danceability,popularity_sub_danceability,popularity_mul_danceability,popularity_bigger_danceability,popularity_add_loudness,popularity_sub_loudness,popularity_mul_loudness,popularity_bigger_loudness,popularity_add_energy,popularity_sub_energy,popularity_mul_energy,popularity_bigger_energy,popularity_add_liveness,popularity_sub_liveness,popularity_mul_liveness,popularity_bigger_liveness,popularity_add_speechiness,popularity_sub_speechiness,popularity_mul_speechiness,popularity_bigger_speechiness,popularity_add_instrumentalness,popularity_sub_instrumentalness,popularity_mul_instrumentalness,popularity_bigger_instrumentalness,popularity_add_tempo_int,popularity_sub_tempo_int,popularity_mul_tempo_int,popularity_bigger_tempo_int,duration_ms_add_acousticness,duration_ms_sub_acousticness,duration_ms_mul_acousticness,duration_ms_bigger_acousticness,duration_ms_add_positiveness,duration_ms_sub_positiveness,duration_ms_mul_positiveness,duration_ms_bigger_positiveness,duration_ms_add_danceability,duration_ms_sub_danceability,duration_ms_mul_danceability,duration_ms_bigger_danceability,duration_ms_add_loudness,duration_ms_sub_loudness,duration_ms_mul_loudness,duration_ms_bigger_loudness,duration_ms_add_energy,duration_ms_sub_energy,duration_ms_mul_energy,duration_ms_bigger_energy,duration_ms_add_liveness,duration_ms_sub_liveness,duration_ms_mul_liveness,duration_ms_bigger_liveness,duration_ms_add_speechiness,duration_ms_sub_speechiness,duration_ms_mul_speechiness,duration_ms_bigger_speechiness,duration_ms_add_instrumentalness,duration_ms_sub_instrumentalness,duration_ms_mul_instrumentalness,duration_ms_bigger_instrumentalness,duration_ms_add_tempo_int,duration_ms_sub_tempo_int,duration_ms_mul_tempo_int,duration_ms_bigger_tempo_int,acousticness_add_positiveness,acousticness_sub_positiveness,acousticness_mul_positiveness,acousticness_bigger_positiveness,acousticness_add_danceability,acousticness_sub_danceability,acousticness_mul_danceability,acousticness_bigger_danceability,acousticness_add_loudness,acousticness_sub_loudness,acousticness_mul_loudness,acousticness_bigger_loudness,acousticness_add_energy,acousticness_sub_energy,acousticness_mul_energy,acousticness_bigger_energy,acousticness_add_liveness,acousticness_sub_liveness,acousticness_mul_liveness,acousticness_bigger_liveness,acousticness_add_speechiness,acousticness_sub_speechiness,acousticness_mul_speechiness,acousticness_bigger_speechiness,acousticness_add_instrumentalness,acousticness_sub_instrumentalness,acousticness_mul_instrumentalness,acousticness_bigger_instrumentalness,acousticness_add_tempo_int,acousticness_sub_tempo_int,acousticness_mul_tempo_int,acousticness_bigger_tempo_int,positiveness_add_danceability,positiveness_sub_danceability,positiveness_mul_danceability,positiveness_bigger_danceability,positiveness_add_loudness,positiveness_sub_loudness,positiveness_mul_loudness,positiveness_bigger_loudness,positiveness_add_energy,positiveness_sub_energy,positiveness_mul_energy,positiveness_bigger_energy,positiveness_add_liveness,positiveness_sub_liveness,positiveness_mul_liveness,positiveness_bigger_liveness,positiveness_add_speechiness,positiveness_sub_speechiness,positiveness_mul_speechiness,positiveness_bigger_speechiness,positiveness_add_instrumentalness,positiveness_sub_instrumentalness,positiven

In [19]:
col_list = [col for col in df_test.columns if col not in [Config.row_id, Config.target]]

pca = PCA()
pca.fit(df_train[col_list])

train_pca = pca.transform(df_train[col_list])
train_pca_cols = ["PCA{}".format(x + 1) for x in range(len(col_list))]
df_train_pca = pd.DataFrame(train_pca, columns=train_pca_cols)

test_pca = pca.transform(df_test[col_list])
test_pca_cols = ["PCA{}".format(x + 1) for x in range(len(col_list))]
df_test_pca = pd.DataFrame(test_pca, columns=test_pca_cols)

display(df_train_pca.head())
display(df_test_pca.head())

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,PCA11,PCA12,PCA13,PCA14,PCA15,PCA16,PCA17,PCA18,PCA19,PCA20,PCA21,PCA22,PCA23,PCA24,PCA25,PCA26,PCA27,PCA28,PCA29,PCA30,PCA31,PCA32,PCA33,PCA34,PCA35,PCA36,PCA37,PCA38,PCA39,PCA40,PCA41,PCA42,PCA43,PCA44,PCA45,PCA46,PCA47,PCA48,PCA49,PCA50,PCA51,PCA52,PCA53,PCA54,PCA55,PCA56,PCA57,PCA58,PCA59,PCA60,PCA61,PCA62,PCA63,PCA64,PCA65,PCA66,PCA67,PCA68,PCA69,PCA70,PCA71,PCA72,PCA73,PCA74,PCA75,PCA76,PCA77,PCA78,PCA79,PCA80,PCA81,PCA82,PCA83,PCA84,PCA85,PCA86,PCA87,PCA88,PCA89,PCA90,PCA91,PCA92,PCA93,PCA94,PCA95,PCA96,PCA97,PCA98,PCA99,PCA100,PCA101,PCA102,PCA103,PCA104,PCA105,PCA106,PCA107,PCA108,PCA109,PCA110,PCA111,PCA112,PCA113,PCA114,PCA115,PCA116,PCA117,PCA118,PCA119,PCA120,PCA121,PCA122,PCA123,PCA124,PCA125,PCA126,PCA127,PCA128,PCA129,PCA130,PCA131,PCA132,PCA133,PCA134,PCA135,PCA136,PCA137,PCA138,PCA139,PCA140,PCA141,PCA142,PCA143,PCA144,PCA145,PCA146,PCA147,PCA148,PCA149,PCA150,PCA151,PCA152,PCA153,PCA154,PCA155,PCA156,PCA157,PCA158,PCA159,PCA160,PCA161,PCA162,PCA163,PCA164,PCA165,PCA166,PCA167,PCA168,PCA169,PCA170,PCA171,PCA172,PCA173,PCA174,PCA175,PCA176,PCA177,PCA178,PCA179,PCA180,PCA181,PCA182,PCA183,PCA184,PCA185,PCA186,PCA187,PCA188,PCA189,PCA190,PCA191,PCA192,PCA193,PCA194,PCA195,PCA196,PCA197,PCA198,PCA199,PCA200,PCA201,PCA202,PCA203,PCA204,PCA205,PCA206,PCA207,PCA208,PCA209,PCA210,PCA211,PCA212,PCA213,PCA214,PCA215,PCA216,PCA217,PCA218,PCA219,PCA220,PCA221,PCA222,PCA223,PCA224,PCA225,PCA226,PCA227,PCA228,PCA229,PCA230,PCA231,PCA232,PCA233,PCA234,PCA235,PCA236,PCA237,PCA238,PCA239,PCA240,PCA241,PCA242,PCA243,PCA244,PCA245,PCA246,PCA247,PCA248,PCA249,PCA250,PCA251,PCA252,PCA253
0,79.075949,-85.613693,204.597951,81.714939,13.522066,123.905658,-16.988429,38.806989,-25.523688,-27.680075,23.408785,-0.043396,-1.136522,0.292510,-0.233483,0.536992,0.133966,-0.246089,-0.092782,-0.175006,0.107648,-0.092135,0.470410,0.549447,0.554570,0.204030,-0.416361,0.501871,0.659917,0.097030,-0.356422,-0.050853,0.366047,0.219096,-0.066796,-0.015161,0.204186,0.049225,-0.009558,0.561381,-0.551603,-0.100749,0.300344,-0.055750,-0.353080,-0.017901,0.577216,0.139959,1.152784,-0.130108,0.349517,-0.260048,0.179588,-0.133860,-0.160508,0.568536,0.260510,0.545593,-0.037463,0.222372,-0.368610,0.148835,-0.621113,0.340916,-0.127101,-0.315085,-0.108848,-0.526355,-0.285044,0.115477,0.003680,0.412452,-0.002793,-0.054495,-0.075432,0.045321,-0.051594,-0.177792,0.118734,-0.373912,0.199677,-0.001341,0.134977,-0.279924,-0.099186,-0.083237,0.005843,-0.051411,-0.123433,-0.139974,0.225509,-0.221866,-0.158542,0.097634,0.281727,0.223173,-0.029075,0.120030,0.105581,0.011631,-0.090868,-0.178213,-0.143162,0.061235,-0.106642,0.011724,-0.000818,0.074127,0.105983,-0.011890,0.091562,0.103253,0.007304,-0.139841,0.163265,-0.070375,-0.081403,-0.202344,0.076322,0.060171,-0.019940,0.022149,0.137727,0.146667,0.061205,-0.021592,0.011424,0.055027,0.070907,0.018470,-0.013462,0.028261,0.022854,0.033734,0.015391,0.002759,0.044091,-0.040897,-0.046742,-0.012931,0.031268,0.037236,-0.006978,-0.036136,0.027319,0.012819,0.000788,-0.011012,-0.037358,0.036072,-0.002806,0.077641,0.014608,0.002464,0.035262,0.013535,0.015458,0.013954,0.043805,-0.063120,-0.007870,0.025049,0.000562,-0.006230,-0.018703,-0.024571,0.013893,-0.038086,0.032113,-0.023197,-0.027951,-0.030899,-0.030806,0.017300,-0.001051,0.018122,6.528491e-14,2.073010e-14,6.760954e-15,-3.076939e-14,-3.961636e-14,3.636484e-14,-3.274723e-17,-4.915373e-15,7.738820e-16,1.701977e-14,1.121889e-14,-1.763089e-14,9.837250e-14,-1.953614e-14,-1.524618e-14,2.503860e-14,1.444708e-14,-5.089656e-14,2.182765e-15,-5.475771e-15,9.831374e-15,1.953643e-15,9.772010e-15,-9.497379e-15,6.384482e-16,-3.345024e-16,-2.076667e-15,-2.889337e-16,4.015155e-16,-1.149493e-15,3.209152e-16,-1.404367e-15,-1.118915e-15,8.670249e-16,-6.436269e-16,7.347419e-16,2.793620e-16,-5.025459e-16,2.859023e-17,1.117027e-16,-4.990226e-16,-4.256834e-16,-3.037828e-18,-7.342540e-17,-5.335723e-16,3.216214e-16,-2.616846e-16,-1.276981e-17,4.9592

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,PCA11,PCA12,PCA13,PCA14,PCA15,PCA16,PCA17,PCA18,PCA19,PCA20,PCA21,PCA22,PCA23,PCA24,PCA25,PCA26,PCA27,PCA28,PCA29,PCA30,PCA31,PCA32,PCA33,PCA34,PCA35,PCA36,PCA37,PCA38,PCA39,PCA40,PCA41,PCA42,PCA43,PCA44,PCA45,PCA46,PCA47,PCA48,PCA49,PCA50,PCA51,PCA52,PCA53,PCA54,PCA55,PCA56,PCA57,PCA58,PCA59,PCA60,PCA61,PCA62,PCA63,PCA64,PCA65,PCA66,PCA67,PCA68,PCA69,PCA70,PCA71,PCA72,PCA73,PCA74,PCA75,PCA76,PCA77,PCA78,PCA79,PCA80,PCA81,PCA82,PCA83,PCA84,PCA85,PCA86,PCA87,PCA88,PCA89,PCA90,PCA91,PCA92,PCA93,PCA94,PCA95,PCA96,PCA97,PCA98,PCA99,PCA100,PCA101,PCA102,PCA103,PCA104,PCA105,PCA106,PCA107,PCA108,PCA109,PCA110,PCA111,PCA112,PCA113,PCA114,PCA115,PCA116,PCA117,PCA118,PCA119,PCA120,PCA121,PCA122,PCA123,PCA124,PCA125,PCA126,PCA127,PCA128,PCA129,PCA130,PCA131,PCA132,PCA133,PCA134,PCA135,PCA136,PCA137,PCA138,PCA139,PCA140,PCA141,PCA142,PCA143,PCA144,PCA145,PCA146,PCA147,PCA148,PCA149,PCA150,PCA151,PCA152,PCA153,PCA154,PCA155,PCA156,PCA157,PCA158,PCA159,PCA160,PCA161,PCA162,PCA163,PCA164,PCA165,PCA166,PCA167,PCA168,PCA169,PCA170,PCA171,PCA172,PCA173,PCA174,PCA175,PCA176,PCA177,PCA178,PCA179,PCA180,PCA181,PCA182,PCA183,PCA184,PCA185,PCA186,PCA187,PCA188,PCA189,PCA190,PCA191,PCA192,PCA193,PCA194,PCA195,PCA196,PCA197,PCA198,PCA199,PCA200,PCA201,PCA202,PCA203,PCA204,PCA205,PCA206,PCA207,PCA208,PCA209,PCA210,PCA211,PCA212,PCA213,PCA214,PCA215,PCA216,PCA217,PCA218,PCA219,PCA220,PCA221,PCA222,PCA223,PCA224,PCA225,PCA226,PCA227,PCA228,PCA229,PCA230,PCA231,PCA232,PCA233,PCA234,PCA235,PCA236,PCA237,PCA238,PCA239,PCA240,PCA241,PCA242,PCA243,PCA244,PCA245,PCA246,PCA247,PCA248,PCA249,PCA250,PCA251,PCA252,PCA253
0,-150.117784,-0.902348,-27.457975,20.232797,29.730243,3.833410,-12.119542,-7.832142,7.384404,2.460822,-24.272722,-0.792923,-0.216964,0.022980,0.686998,0.978805,-0.232045,0.241337,-0.244497,0.108931,0.073305,0.763736,-0.254082,-0.859562,0.266217,-0.020358,0.055985,-0.040913,0.428080,0.170069,-0.026824,-0.476582,-0.216556,-0.774301,0.808675,0.293340,-0.242342,-0.116361,-0.084126,0.087687,-0.187060,-0.033905,-0.328462,-0.003944,0.008570,0.096342,-0.196362,0.547993,0.210611,0.171458,-0.216014,-0.043690,0.094000,0.204228,-0.033860,-0.076303,-0.014236,0.097747,-0.126635,0.075857,0.002775,0.078384,-0.101211,0.003870,0.170745,-0.125703,-0.027837,0.102900,-0.294992,-0.247074,0.423412,0.467431,0.070275,-0.128159,0.402015,0.152361,-0.193374,-0.069341,0.219427,0.042050,-0.004116,0.053645,-0.122860,-0.052867,-0.081564,0.037527,0.022541,-0.066787,-0.011986,0.030605,-0.046804,0.132403,0.001269,0.055082,-0.012823,-0.028916,0.001318,0.075377,-0.001356,0.050791,-0.053312,-0.108513,0.056427,0.023055,-0.019882,0.014253,0.058883,-0.134141,-0.004762,0.013348,-0.028060,-0.005754,-0.039463,0.003521,-0.056032,0.018084,-0.056299,0.110963,-0.019051,-0.094592,0.148015,0.026575,0.137859,0.047334,0.033501,0.039061,-0.040790,0.001286,-0.021627,-0.050154,0.026957,0.026022,0.015290,-0.018536,0.000213,0.011770,0.020673,-0.002770,-0.037321,-0.032370,0.039700,-0.051688,-0.004088,-0.008003,0.014561,0.019134,-0.014963,0.006578,-0.004630,-0.004281,0.009386,-0.022831,0.040191,0.019318,-0.040928,-0.042984,-0.010973,-0.021735,-0.029238,-0.010340,-0.021432,-0.012617,0.012194,-0.003518,0.007538,-0.006653,-0.010553,0.016472,0.007208,0.002705,-0.004797,-0.003768,0.002165,0.001489,-0.006333,0.013319,-3.730883e-14,2.776375e-14,-1.853439e-14,2.796438e-14,3.746263e-14,2.553269e-14,-1.556833e-14,-1.249935e-14,3.036837e-14,-1.460103e-14,-6.167068e-15,-5.910303e-15,3.792765e-16,3.861203e-15,1.022599e-14,5.253307e-15,9.039543e-15,1.693124e-14,1.480898e-14,5.425340e-15,-1.718776e-15,-1.898132e-15,-7.052939e-16,4.104618e-15,-8.608716e-16,9.799988e-17,1.295835e-16,-9.584533e-17,-2.926220e-16,6.191578e-16,4.639178e-16,-2.688813e-16,2.532896e-16,-4.168564e-16,3.570787e-16,1.592207e-16,-2.786314e-16,-1.695944e-16,-2.022287e-16,-5.818728e-17,1.201618e-16,6.308853e-16,-1.697259e-16,-4.076446e-17,-7.512127e-17,1.407728e-17,5.637797e-17,9.644567e-17,-1.2639

In [20]:
train_exp_df = pd.DataFrame(pca.explained_variance_ratio_, index=train_pca_cols)
display(train_exp_df.head(30))
display(train_exp_df.head(30).sum())

,0
PCA1,0.472057
PCA2,0.150469
PCA3,0.109152
PCA4,0.050831
PCA5,0.046546
PCA6,0.041831
PCA7,0.034441
PCA8,0.033810
PCA9,0.025316
PCA10,0.021926


0    0.999904
dtype: float64

In [21]:
df_train = pd.concat(objs=[df_train, df_train_pca[["PCA{}".format(x + 1) for x in range(30)]]], axis=1)

df_test = df_test.reset_index()
df_test = pd.concat(objs=[df_test, df_test_pca[["PCA{}".format(x + 1) for x in range(30)]]], axis=1)
df_test = df_test.set_index('level_0')
display(df_train.head())
display(df_test.head())

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo_int,region_A,region_B,region_C,region_D,region_E,region_F,region_G,region_H,region_I,region_J,region_K,region_L,region_M,region_N,region_O,region_P,region_Q,region_R,region_S,region_T,unknown,duration_long,popularity_add_duration_ms,popularity_sub_duration_ms,popularity_mul_duration_ms,popularity_bigger_duration_ms,popularity_add_acousticness,popularity_sub_acousticness,popularity_mul_acousticness,popularity_bigger_acousticness,popularity_add_positiveness,popularity_sub_positiveness,popularity_mul_positiveness,popularity_bigger_positiveness,popularity_add_danceability,popularity_sub_danceability,popularity_mul_danceability,popularity_bigger_danceability,popularity_add_loudness,popularity_sub_loudness,popularity_mul_loudness,popularity_bigger_loudness,popularity_add_energy,popularity_sub_energy,popularity_mul_energy,popularity_bigger_energy,popularity_add_liveness,popularity_sub_liveness,popularity_mul_liveness,popularity_bigger_liveness,popularity_add_speechiness,popularity_sub_speechiness,popularity_mul_speechiness,popularity_bigger_speechiness,popularity_add_instrumentalness,popularity_sub_instrumentalness,popularity_mul_instrumentalness,popularity_bigger_instrumentalness,popularity_add_tempo_int,popularity_sub_tempo_int,popularity_mul_tempo_int,popularity_bigger_tempo_int,duration_ms_add_acousticness,duration_ms_sub_acousticness,duration_ms_mul_acousticness,duration_ms_bigger_acousticness,duration_ms_add_positiveness,duration_ms_sub_positiveness,duration_ms_mul_positiveness,duration_ms_bigger_positiveness,duration_ms_add_danceability,duration_ms_sub_danceability,duration_ms_mul_danceability,duration_ms_bigger_danceability,duration_ms_add_loudness,duration_ms_sub_loudness,duration_ms_mul_loudness,duration_ms_bigger_loudness,duration_ms_add_energy,duration_ms_sub_energy,duration_ms_mul_energy,duration_ms_bigger_energy,duration_ms_add_liveness,duration_ms_sub_liveness,duration_ms_mul_liveness,duration_ms_bigger_liveness,duration_ms_add_speechiness,duration_ms_sub_speechiness,duration_ms_mul_speechiness,duration_ms_bigger_speechiness,duration_ms_add_instrumentalness,duration_ms_sub_instrumentalness,duration_ms_mul_instrumentalness,duration_ms_bigger_instrumentalness,duration_ms_add_tempo_int,duration_ms_sub_tempo_int,duration_ms_mul_tempo_int,duration_ms_bigger_tempo_int,acousticness_add_positiveness,acousticness_sub_positiveness,acousticness_mul_positiveness,acousticness_bigger_positiveness,acousticness_add_danceability,acousticness_sub_danceability,acousticness_mul_danceability,acousticness_bigger_danceability,acousticness_add_loudness,acousticness_sub_loudness,acousticness_mul_loudness,acousticness_bigger_loudness,acousticness_add_energy,acousticness_sub_energy,acousticness_mul_energy,acousticness_bigger_energy,acousticness_add_liveness,acousticness_sub_liveness,acousticness_mul_liveness,acousticness_bigger_liveness,acousticness_add_speechiness,acousticness_sub_speechiness,acousticness_mul_speechiness,acousticness_bigger_speechiness,acousticness_add_instrumentalness,acousticness_sub_instrumentalness,acousticness_mul_instrumentalness,acousticness_bigger_instrumentalness,acousticness_add_tempo_int,acousticness_sub_tempo_int,acousticness_mul_tempo_int,acousticness_bigger_tempo_int,positiveness_add_danceability,positiveness_sub_danceability,positiveness_mul_danceability,positiveness_bigger_danceability,positiveness_add_loudness,positiveness_sub_loudness,positiveness_mul_loudness,positiveness_bigger_loudness,positiveness_add_energy,positiveness_sub_energy,positiveness_mul_energy,positiveness_bigger_energy,positiveness_add_liveness,positiveness_sub_liveness,positiveness_mul_liveness,positiveness_bigger_liveness,positiveness_add_speechiness,positiveness_sub_speechiness,positiveness_mul_speechiness,positiveness_bigger_speechiness,positiveness_add_instrumentalness,positiveness_sub_instrumentalness,pos

,index,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo_int,region_A,region_B,region_C,region_D,region_E,region_F,region_G,region_H,region_I,region_J,region_K,region_L,region_M,region_N,region_O,region_P,region_Q,region_R,region_S,region_T,unknown,duration_long,popularity_add_duration_ms,popularity_sub_duration_ms,popularity_mul_duration_ms,popularity_bigger_duration_ms,popularity_add_acousticness,popularity_sub_acousticness,popularity_mul_acousticness,popularity_bigger_acousticness,popularity_add_positiveness,popularity_sub_positiveness,popularity_mul_positiveness,popularity_bigger_positiveness,popularity_add_danceability,popularity_sub_danceability,popularity_mul_danceability,popularity_bigger_danceability,popularity_add_loudness,popularity_sub_loudness,popularity_mul_loudness,popularity_bigger_loudness,popularity_add_energy,popularity_sub_energy,popularity_mul_energy,popularity_bigger_energy,popularity_add_liveness,popularity_sub_liveness,popularity_mul_liveness,popularity_bigger_liveness,popularity_add_speechiness,popularity_sub_speechiness,popularity_mul_speechiness,popularity_bigger_speechiness,popularity_add_instrumentalness,popularity_sub_instrumentalness,popularity_mul_instrumentalness,popularity_bigger_instrumentalness,popularity_add_tempo_int,popularity_sub_tempo_int,popularity_mul_tempo_int,popularity_bigger_tempo_int,duration_ms_add_acousticness,duration_ms_sub_acousticness,duration_ms_mul_acousticness,duration_ms_bigger_acousticness,duration_ms_add_positiveness,duration_ms_sub_positiveness,duration_ms_mul_positiveness,duration_ms_bigger_positiveness,duration_ms_add_danceability,duration_ms_sub_danceability,duration_ms_mul_danceability,duration_ms_bigger_danceability,duration_ms_add_loudness,duration_ms_sub_loudness,duration_ms_mul_loudness,duration_ms_bigger_loudness,duration_ms_add_energy,duration_ms_sub_energy,duration_ms_mul_energy,duration_ms_bigger_energy,duration_ms_add_liveness,duration_ms_sub_liveness,duration_ms_mul_liveness,duration_ms_bigger_liveness,duration_ms_add_speechiness,duration_ms_sub_speechiness,duration_ms_mul_speechiness,duration_ms_bigger_speechiness,duration_ms_add_instrumentalness,duration_ms_sub_instrumentalness,duration_ms_mul_instrumentalness,duration_ms_bigger_instrumentalness,duration_ms_add_tempo_int,duration_ms_sub_tempo_int,duration_ms_mul_tempo_int,duration_ms_bigger_tempo_int,acousticness_add_positiveness,acousticness_sub_positiveness,acousticness_mul_positiveness,acousticness_bigger_positiveness,acousticness_add_danceability,acousticness_sub_danceability,acousticness_mul_danceability,acousticness_bigger_danceability,acousticness_add_loudness,acousticness_sub_loudness,acousticness_mul_loudness,acousticness_bigger_loudness,acousticness_add_energy,acousticness_sub_energy,acousticness_mul_energy,acousticness_bigger_energy,acousticness_add_liveness,acousticness_sub_liveness,acousticness_mul_liveness,acousticness_bigger_liveness,acousticness_add_speechiness,acousticness_sub_speechiness,acousticness_mul_speechiness,acousticness_bigger_speechiness,acousticness_add_instrumentalness,acousticness_sub_instrumentalness,acousticness_mul_instrumentalness,acousticness_bigger_instrumentalness,acousticness_add_tempo_int,acousticness_sub_tempo_int,acousticness_mul_tempo_int,acousticness_bigger_tempo_int,positiveness_add_danceability,positiveness_sub_danceability,positiveness_mul_danceability,positiveness_bigger_danceability,positiveness_add_loudness,positiveness_sub_loudness,positiveness_mul_loudness,positiveness_bigger_loudness,positiveness_add_energy,positiveness_sub_energy,positiveness_mul_energy,positiveness_bigger_energy,positiveness_add_liveness,positiveness_sub_liveness,positiveness_mul_liveness,positiveness_bigger_liveness,positiveness_add_speechiness,positiveness_sub_speechiness,positiveness_mul_speechiness,positiveness_bigger_speechiness,positiveness_add_instrumentalness,positiveness_sub_instrumentalness,positiven

## Save Dataset

In [22]:
Config.NB

'103'

In [23]:
pd.to_pickle(df_train, Config.processed_data_dir + f"nb{Config.NB}_train.pkl", compression='zip')
pd.to_pickle(df_test, Config.processed_data_dir + f"nb{Config.NB}_test.pkl", compression='zip')

## 検証メモ